In [1]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf



In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int

In [3]:
from kidneyClassifier import *
from kidneyClassifier.constants import *
from kidneyClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = Path(CONFIG_FILE_PATH),
            params_filepath = Path(PARAMS_FILE_PATH)):
                
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                
                create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=config.root_dir,
                base_model_path=config.base_model_path,
                updated_base_model_path = config.updated_base_model_path,
                params_image_size=self.params.IMAGE_SIZE,
                params_learning_rate =self.params.LEARNING_RATE,
                params_include_top=self.params.INCLUDE_TOP,
                params_weight=self.params.WEIGHTS,
                params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [5]:
from kidneyClassifier.constants import *

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weight,
            input_shape=self.config.params_image_size,
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif(freeze_till is not None) and (freeze_all > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            input= model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate-learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(Path(path))

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

  

In [6]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-08-28 01:55:32,964: INFO: common: yaml file: ..\config\config.yaml loaded successfully]
[2024-08-28 01:55:32,976: INFO: common: yaml file: ..\params.yaml loaded successfully]
[2024-08-28 01:55:32,976: INFO: common: created directory at: artifacts]
[2024-08-28 01:55:32,984: INFO: common: created directory at: artifacts/prepare_base_model]


ValueError: Unrecognized keyword arguments passed to Model: {'input': <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor>, 'outputs': <KerasTensor shape=(None, 2), dtype=float32, sparse=False, name=keras_tensor_20>}